FIAP - Tecnólogo de Inteligência Artificial

# Checkpoint 05

Faça o exercício abaixo <font color="red">em grupos de até 4 pessoas</font>, para praticar o uso do `sklearn avançado` em Machine Learning. Utilize os conceitos vistos em aula, e consulte [documentações](https://scikit-learn.org/stable/) na internet. Em caso de dúvidas, contate o Prof. Michel.

**Entrega**: 25/setembro/2023, via Teams.

**Apresentação**: utilizaremos a aula de 25/setembro para apresentação dos trabalhos e rodada de feedbacks!


## Apresentação do trabalho

Neste checkpoint vamos trabalhar com modelagem de dados utilizando os recursos avançados do `sklearn`.

Isso implica o uso de:  

- Tratamento de missing
- Identificação e tratamento de outliers
- Feature engineering (redução de dimensionalidade, padronização, encoder, etc)
- Lidar com desbalanceamento de dados
- Seleção de modelos pertinentes ao problema elencado (regressão x classificação)
- Otimização de hiperparâmetros
- Validação cruzada
- Escolha de métricas pertinentes
- Pipelines
- Interpretabilidade dos resultados

**Dataset**: vocês estão livres para escolher um dataset de interesse do grupo. Com o que gostariam de trabalhar? Saúde? Varejo? Jogos? Agronegócio? Finanças? Escolham uma base pública do Kaggle e sejam felizes. =)

**Critérios de avaliação**: A correção do checkpoint será feita mediante os seguintes critérios:
- completude do pipeline
- emprego correto dos conceitos de machine learning vistos em sala de aula
- apresentação do dia 25/setembro (análise/justificativas dos resultados)


In [ ]:
#############################
#                           #
#   Participantes do grupo  #
#                           #
#############################

# Arthur Hieda Cunha - 551882

# Henrique Marra Barbosa - 97672

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline

#montar o Google Drive
drive.mount('/content/drive')

#caminho para o arquivo 'images.csv' no Google Drive
file_path = "/content/drive/MyDrive/Machine Learning/images.csv"

#carregar o conjunto de dados
data = pd.read_csv(file_path)

#visualizar as primeiras linhas do conjunto de dados
print(data.head())

#identificar e tratar valores ausentes
numeric_columns = data.select_dtypes(include=[np.number]).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

#identificar e tratar outliers (por exemplo, usando Z-score)
z_scores = np.abs(zscore(data[numeric_columns]))
data_no_outliers = data[(z_scores < 3).all(axis=1)]  # Remove outliers

#redução de dimensionalidade (PCA)
numeric_columns = data.select_dtypes(include=[np.number]).columns

#verifica se há algum valor nulo nas colunas numéricas
if data[numeric_columns].isnull().values.any():
    print("Existem valores nulos nas colunas numéricas. Preencha ou remova esses valores antes de prosseguir.")
else:
    #selecione apenas as colunas numéricas
    data_numeric = data[numeric_columns]

    #aplica o PCA apenas se houver colunas numéricas
    if data_numeric.shape[1] > 0:
        #ajuste para o número real de features
        n_components = min(1, data_numeric.shape[1])

        pca = PCA(n_components=n_components)  #ajuste aqui
        data_pca = pca.fit_transform(data_numeric)

        #visualiza as primeiras linhas do resultado do PCA
        print("Resultados do PCA:")
        print(data_pca)
    else:
        print("Não há colunas numéricas para aplicar o PCA.")

#padronização apenas nas colunas numéricas
numeric_columns = data.select_dtypes(include=[np.number]).columns
data_numeric = data[numeric_columns]

#verifica se há algum valor nulo nas colunas numéricas
if data_numeric.isnull().values.any():
    print("Existem valores nulos nas colunas numéricas. Preencha ou remova esses valores antes de prosseguir.")
else:
    #padronização
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_numeric)

    #visualiza as primeiras linhas do resultado da padronização
    print("Resultados da padronização:")
    print(data_scaled)

#encder (para variáveis categóricas)
encoder = LabelEncoder()

#verifica se a coluna 'label' é categórica e aplica o encoder
if 'label' in data.columns:
    data['label_encoded'] = encoder.fit_transform(data['label'])
else:
    print("A coluna 'label' não foi encontrada no conjunto de dados.")

#lidar com desbalanceamento usando técnicas de oversampling ou undersampling
if 'label_encoded' in data.columns:
    #features numéricas para o SMOTE
    numeric_columns = data.select_dtypes(include=[np.number]).columns
    data_numeric = data[numeric_columns]

    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(data_numeric, data['label_encoded'])
else:
    print("A coluna 'label_encoded' não foi encontrada no conjunto de dados.")

#dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# ...

#otimização de hiperparâmetros usando Grid Search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_model_logistic = grid_search.best_estimator_

#validar o modelo usando validação cruzada
cv_scores = cross_val_score(best_model_logistic, X_test, y_test, cv=5)
print("Cross-validation scores for Logistic Regression:", cv_scores)

#escolher métricas apropriadas para o tipo de problema
#classificação
y_pred_logistic = best_model_logistic.predict(X_test)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)

#regressão
mse_logistic = mean_squared_error(y_test, y_pred_logistic)

#construir um pipeline para encapsular as etapas de pré-processamento e modelo
pipeline_logistic = Pipeline([
    ('scaling', StandardScaler()),
    ('model', LogisticRegression())
])

pipeline_logistic.fit(X_train, y_train)

#interpretação do modelo (exemplo: coeficientes para a regressão logística)
coefficients_logistic = best_model_logistic.coef_
print("Coefficients for Logistic Regression:", coefficients_logistic)

Mounted at /content/drive
                                  image  sender_id     label   kids
0  4285fab0-751a-4b74-8e9b-43af05deee22        124  Not sure  False
1  ea7b6656-3f84-4eb3-9099-23e623fc1018        148   T-Shirt  False
2  00627a3f-0477-401c-95eb-92642cbe078d         94  Not sure  False
3  ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa         43   T-Shirt  False
4  3b86d877-2b9e-4c8b-a6a2-1d87513309d0        189     Shoes  False
Resultados do PCA:
[[-58.9496576]
 [-34.9496576]
 [-88.9496576]
 ...
 [127.0503424]
 [ 21.0503424]
 [ 21.0503424]]
Resultados da padronização:
[[-0.59289713]
 [-0.35151267]
 [-0.8946277 ]
 ...
 [ 1.27783242]
 [ 0.21171773]
 [ 0.21171773]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Cross-validation scores for Logistic Regression: [0.18665019 0.15574784 0.17799753 0.21508035 0.19430693]
Coefficients for Logistic Regression: [[ 0.01550599 -0.7997787 ]
 [ 0.01475821 -0.63680132]
 [ 0.01518146 -0.55829044]
 [ 0.01031041 -0.29598669]
 [ 0.00809252 -0.18071804]
 [ 0.00612121 -0.096861  ]
 [ 0.00346058 -0.01482048]
 [ 0.00117745  0.05370568]
 [ 0.00181204  0.05305161]
 [-0.00148569  0.12252871]
 [-0.00280538  0.14820299]
 [-0.00500681  0.18231354]
 [-0.00204123  0.15083592]
 [-0.00483098  0.19634555]
 [-0.00714851  0.23882868]
 [-0.0053228   0.22033341]
 [-0.01018824  0.28430447]
 [-0.01025492  0.28653376]
 [-0.01386097  0.32080778]
 [-0.01347435  0.32546457]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
